# ETF Portfolio Optimization - Baseline Demo

This notebook demonstrates the baseline portfolio optimization system with:
- Data loading and preprocessing
- Feature engineering
- Baseline strategies (Equal Weight, Mean-Variance)
- Backtesting and performance evaluation
- Comprehensive visualizations

In [ ]:
# Add src to path
import sys
sys.path.append('../src')

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Project imports
from data import ETFDataLoader, load_default_etfs
from features import FeatureEngineer, create_feature_summary
from strategies import EqualWeightStrategy, MeanVarianceStrategy, create_60_40_strategy
from backtest import PortfolioBacktest, compare_strategies
from metrics import calculate_all_metrics, compare_strategies as compare_metrics, format_metrics_table
from visualization import (
    plot_equity_curves, plot_drawdown, plot_multiple_drawdowns,
    plot_allocation_over_time, plot_correlation_matrix,
    plot_rolling_sharpe, create_performance_dashboard
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

## 1. Load Data

Load historical price data for VTI, BND, and QQQ ETFs.

In [ ]:
# Load ETF data (2015-2025)
prices = load_default_etfs(start_date='2015-01-01')

print(f"\nLoaded {len(prices)} days of data")
print(f"ETFs: {', '.join(prices.columns)}")
print(f"Date range: {prices.index[0].date()} to {prices.index[-1].date()}")

In [ ]:
# Show summary statistics
loader = ETFDataLoader()
summary = loader.get_data_summary(prices)
print("\n" + "="*80)
print("ETF Summary Statistics")
print("="*80)
print(summary)

In [ ]:
# Visualize price history
fig, ax = plt.subplots(figsize=(14, 6))
normalized_prices = prices / prices.iloc[0] * 100
normalized_prices.plot(ax=ax, linewidth=2)
ax.set_title('ETF Price History (Normalized to 100)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Normalized Price', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Split Data

Split data into train, validation, and test sets:
- **Train**: 2015-2021 (used for strategy development)
- **Validation**: 2022 (used for hyperparameter tuning)
- **Test**: 2023-2025 (held-out for final evaluation)

In [ ]:
# Split data
train, val, test = loader.split_train_val_test(prices)

print("\n" + "="*80)
print("Training Set Summary")
print("="*80)
print(loader.get_data_summary(train))

## 3. Feature Engineering

Compute features for portfolio optimization:
- Daily returns
- Rolling volatility (30-day)
- Rolling momentum (30-day)
- Rolling Sharpe ratio
- Average correlation with other assets

In [ ]:
# Create feature engineer
feature_eng = FeatureEngineer(lookback_window=30)

# Compute features on training data
features_train = feature_eng.compute_all_features(train)

print(f"\nComputed {len(features_train.columns)} features")
print(f"Feature columns: {', '.join(features_train.columns[:10])}...")

In [ ]:
# Show feature summary
print("\n" + "="*80)
print("Feature Summary (Training Data)")
print("="*80)
print(create_feature_summary(features_train))

In [ ]:
# Visualize correlations
returns_train = loader.calculate_returns(train).dropna()
plot_correlation_matrix(returns_train, title='Training Set: Asset Correlation Matrix')
plt.show()

## 4. Define Baseline Strategies

We'll compare three baseline strategies:
1. **Equal Weight**: Simple 1/N allocation
2. **Mean-Variance**: Maximum Sharpe ratio optimization using historical data
3. **60/40**: Traditional 60% stocks / 40% bonds

In [ ]:
# Define strategies
strategies = {
    'Equal Weight': EqualWeightStrategy(),
    'Mean-Variance (Max Sharpe)': MeanVarianceStrategy(lookback_days=252, method='max_sharpe'),
    '60/40 Portfolio': create_60_40_strategy(['VTI', 'QQQ'], ['BND'])
}

In [ ]:
# Show initial allocations
print("\n" + "="*80)
print("Initial Allocations (using full training data)")
print("="*80)

for name, strategy in strategies.items():
    weights = strategy.allocate(train)
    print(f"\n{name}:")
    for ticker, weight in weights.items():
        print(f"  {ticker}: {weight:>6.2%}")

## 5. Backtest on Training Data

Run backtest with monthly rebalancing and 0.1% transaction costs.

In [ ]:
# Run backtest on training data
print("\n" + "="*80)
print("Backtesting on Training Data (2015-2021)")
print("="*80)

results_train, allocations_train = compare_strategies(
    strategies,
    train,
    initial_capital=100000,
    transaction_cost=0.001,  # 0.1%
    rebalance_frequency='M'   # Monthly
)

In [ ]:
# Calculate metrics
metrics_train = compare_metrics(results_train, allocations_train)
metrics_train_formatted = format_metrics_table(metrics_train)

print("\n" + "="*80)
print("Training Set Performance Metrics")
print("="*80)
print(metrics_train_formatted.to_string())

In [ ]:
# Visualize equity curves
plot_equity_curves(results_train, title='Training Set: Portfolio Performance')
plt.show()

In [ ]:
# Visualize drawdowns
plot_multiple_drawdowns(results_train, title='Training Set: Drawdown Comparison')
plt.show()

In [ ]:
# Visualize allocation dynamics for Mean-Variance strategy
if 'Mean-Variance (Max Sharpe)' in allocations_train:
    plot_allocation_over_time(
        allocations_train['Mean-Variance (Max Sharpe)'],
        title='Mean-Variance Strategy: Allocation Over Time (Training)'
    )
    plt.show()

## 6. Backtest on Validation Data

Evaluate strategies on validation set (2022).

In [ ]:
# Run backtest on validation data
print("\n" + "="*80)
print("Backtesting on Validation Data (2022)")
print("="*80)

results_val, allocations_val = compare_strategies(
    strategies,
    val,
    initial_capital=100000,
    transaction_cost=0.001,
    rebalance_frequency='M'
)

In [ ]:
# Calculate metrics
metrics_val = compare_metrics(results_val, allocations_val)
metrics_val_formatted = format_metrics_table(metrics_val)

print("\n" + "="*80)
print("Validation Set Performance Metrics")
print("="*80)
print(metrics_val_formatted.to_string())

In [ ]:
# Visualize validation results
plot_equity_curves(results_val, title='Validation Set: Portfolio Performance')
plt.show()

## 7. Backtest on Test Data

Final evaluation on held-out test set (2023-2025).

In [ ]:
# Run backtest on test data
print("\n" + "="*80)
print("Backtesting on Test Data (2023-2025)")
print("="*80)

results_test, allocations_test = compare_strategies(
    strategies,
    test,
    initial_capital=100000,
    transaction_cost=0.001,
    rebalance_frequency='M'
)

In [ ]:
# Calculate metrics
metrics_test = compare_metrics(results_test, allocations_test)
metrics_test_formatted = format_metrics_table(metrics_test)

print("\n" + "="*80)
print("Test Set Performance Metrics")
print("="*80)
print(metrics_test_formatted.to_string())

In [ ]:
# Visualize test results
plot_equity_curves(results_test, title='Test Set: Portfolio Performance')
plt.show()

In [ ]:
# Create comprehensive dashboard for test set
create_performance_dashboard(
    results_test,
    allocations_test,
    metrics_test
)
plt.show()

## 8. Cross-Period Comparison

Compare strategy performance across train/val/test periods.

In [ ]:
# Create comparison table
comparison_data = []

for strategy_name in strategies.keys():
    comparison_data.append({
        'Strategy': strategy_name,
        'Period': 'Train',
        'Sharpe': metrics_train.loc[strategy_name, 'Sharpe Ratio'],
        'Return (%)': metrics_train.loc[strategy_name, 'Annualized Return'] * 100,
        'Volatility (%)': metrics_train.loc[strategy_name, 'Annualized Volatility'] * 100,
        'Max DD (%)': metrics_train.loc[strategy_name, 'Max Drawdown'] * 100
    })
    comparison_data.append({
        'Strategy': strategy_name,
        'Period': 'Val',
        'Sharpe': metrics_val.loc[strategy_name, 'Sharpe Ratio'],
        'Return (%)': metrics_val.loc[strategy_name, 'Annualized Return'] * 100,
        'Volatility (%)': metrics_val.loc[strategy_name, 'Annualized Volatility'] * 100,
        'Max DD (%)': metrics_val.loc[strategy_name, 'Max Drawdown'] * 100
    })
    comparison_data.append({
        'Strategy': strategy_name,
        'Period': 'Test',
        'Sharpe': metrics_test.loc[strategy_name, 'Sharpe Ratio'],
        'Return (%)': metrics_test.loc[strategy_name, 'Annualized Return'] * 100,
        'Volatility (%)': metrics_test.loc[strategy_name, 'Annualized Volatility'] * 100,
        'Max DD (%)': metrics_test.loc[strategy_name, 'Max Drawdown'] * 100
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("Cross-Period Performance Comparison")
print("="*80)
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize Sharpe ratio across periods
fig, ax = plt.subplots(figsize=(12, 6))

for strategy_name in strategies.keys():
    strategy_data = comparison_df[comparison_df['Strategy'] == strategy_name]
    ax.plot(strategy_data['Period'], strategy_data['Sharpe'], 
            marker='o', linewidth=2, markersize=8, label=strategy_name)

ax.set_xlabel('Period', fontsize=12)
ax.set_ylabel('Sharpe Ratio', fontsize=12)
ax.set_title('Sharpe Ratio Across Periods', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
plt.tight_layout()
plt.show()

## 9. Key Findings and Next Steps

### Observations:
1. Compare baseline strategies across different market conditions
2. Identify which strategy performs best in terms of risk-adjusted returns
3. Note consistency (or lack thereof) across train/val/test periods

### Next Steps:
1. **Hyperparameter tuning**: Experiment with different lookback windows, rebalancing frequencies
2. **Additional features**: Add regime detection, macro indicators
3. **More baselines**: Risk parity, minimum variance, robust optimization
4. **RL implementation**: Build custom environment and train RL agents with Sharpe-aware reward
5. **Walk-forward validation**: Implement rolling window backtesting
6. **Transaction cost sensitivity**: Test impact of different cost assumptions

## 10. Export Results

Save results for future reference.

In [ ]:
# Save metrics to CSV
metrics_train_formatted.to_csv('../data/metrics_train.csv')
metrics_val_formatted.to_csv('../data/metrics_val.csv')
metrics_test_formatted.to_csv('../data/metrics_test.csv')
comparison_df.to_csv('../data/comparison_summary.csv', index=False)

print("✓ Results saved to data/ directory")